## Task 3: Exploration and Clustering Neighborhoods in Toronto

Week 3: Applied Data Science Capstone

In [32]:
# import modules
import pandas as pd
import requests
import numpy as np
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
# load data from task 2
geo_data = pd.read_csv('data_2.csv')
del geo_data['Unnamed: 0']
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


**Using Foursquare API to explore and segment neighborhoods**

In [7]:
CLIENT_ID = 'Hidden Content' # your Foursquare ID
CLIENT_SECRET = 'Hidden Content' # your Foursquare Secret
VERSION = 'Hidden Content'

**Extract Borough containing Toronto**

In [5]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.678148,-79.295349
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578
43,M4M,East Toronto,Studio District,43.648000,-79.339260
44,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415


**Get Nearby Venues**

In [8]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ro

In [12]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.678148,-79.295349,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.678148,-79.295349,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.678148,-79.295349,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.678148,-79.295349,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
4,The Beaches,43.678148,-79.295349,Upper Beaches,43.680563,-79.292869,Neighborhood


In [13]:
# count venues
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,64,64,64,64,64,64
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst\n Quay, South Niagara, Island airport",67,67,67,67,67,67
Central Bay Street,83,83,83,83,83,83
Christie,13,13,13,13,13,13
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns  = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1672, 230)

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.0,0.000000,0.0,0.000000,0.015625,0.000000,0.000000,0.0,0.015625,...,0.000000,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.022727,0.0,0.000000,0.022727,0.022727,0.000000,0.0,0.000000,...,0.045455,0.022727,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,Business reply mail Processing CentrE,0.0,0.000000,0.0,0.030000,0.010000,0.010000,0.020000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.020000,0.000000,0.000000,0.010000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.014925,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,Central Bay Street,0.0,0.000000,0.0,0.012048,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.012048,0.0,0.0,0.000000,0.012048,0.012048,0.012048,0.0


In [18]:
# find most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Restaurant,Hotel,Breakfast Spot,Lounge,Cheese Shop,Bakery,Café
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Restaurant,Gift Shop,Thrift / Vintage Store,Lounge,Pet Store,Cocktail Bar,Boutique,Brewery
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Restaurant,Bar,Pub,American Restaurant,Café,Seafood Restaurant,Italian Restaurant,Concert Hall
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Café,Gym / Fitness Center,Park,French Restaurant,Italian Restaurant,Speakeasy,Bar,Lounge
4,Central Bay Street,Coffee Shop,Clothing Store,Japanese Restaurant,Tea Room,Middle Eastern Restaurant,Electronics Store,Plaza,Fast Food Restaurant,Italian Restaurant,Restaurant


**K-Means to Cluster Neighborhoods into 10 clusters**

In [25]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Coffee Shop,Trail,Church,Pub,Women's Store,Farm,Elementary School,Ethiopian Restaurant,Falafel Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564,2,Bus Line,Business Service,Park,Discount Store,Grocery Store,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578,0,Sandwich Place,Pet Store,Italian Restaurant,Park,Pizza Place,Coffee Shop,Pub,Movie Theater,Restaurant,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.648000,-79.339260,7,Baseball Field,Night Market,Business Service,Government Building,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop
44,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415,1,Bus Line,Swim School,Lawyer,Women's Store,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market


In [27]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Restaurant,Hotel,Breakfast Spot,Lounge,Cheese Shop,Bakery,Café
1,0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Restaurant,Gift Shop,Thrift / Vintage Store,Lounge,Pet Store,Cocktail Bar,Boutique,Brewery
2,0,Business reply mail Processing CentrE,Coffee Shop,Hotel,Restaurant,Bar,Pub,American Restaurant,Café,Seafood Restaurant,Italian Restaurant,Concert Hall
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Café,Gym / Fitness Center,Park,French Restaurant,Italian Restaurant,Speakeasy,Bar,Lounge
4,0,Central Bay Street,Coffee Shop,Clothing Store,Japanese Restaurant,Tea Room,Middle Eastern Restaurant,Electronics Store,Plaza,Fast Food Restaurant,Italian Restaurant,Restaurant


In [34]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto, CA are {latitude}, {longitude}.')

The geograpical coordinate of Toronto, CA are 43.6534817, -79.3839347.


**Create a map of Toronto with neighborhoods on top**

In [35]:
# create map 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto